In [3]:
from data_pre_processing_and_transform.create_torch_datset import TorchDataModule, TorchDatasetTS
from data_pre_processing_and_transform.data_preprocessing import DataPreprocessing

import os
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from collections import defaultdict



tqdm.pandas() 



In [5]:
class LSTMModel(nn.Module):

    def __init__(self, n_features, sequence_length, number_of_dependent_signals, n_hidden=64, n_layers = 2):
        super().__init__()

        self.n_hidden = n_hidden
        self.sequence_length = sequence_length
        self.number_of_dependent_signals = number_of_dependent_signals

        """self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            batch_first=True,
            num_layers=  n_layers,
            dropout=0.2)
            """
        
        self.lstm1 = nn.LSTM(input_size = n_features,
                            hidden_size = n_hidden,
                            num_layers=n_layers,
                            batch_first=True,
                            dropout=0.2)
        
        #self.bn1 = nn.BatchNorm1d()



        self.regressor1 = nn.Linear(n_hidden, sequence_length * number_of_dependent_signals)

    def forward(self, x):
        #self.lstm.flatten_prameters()  # Even if we don't use flatten prameter the code will work. But it helps in distributed training of GPU

        #out, (hidden, _) = self.lstm(x)
        #out = hidden[-1]

        out, (_, _) = self.lstm1(x)
        out = F.relu(out)
        
        return self.regressor1(out)
    

class Predictor(pl.LightningModule):
    def __init__(self, n_features, sequence_length, number_of_dependent_signals):
        super().__init__()
        self.model = LSTMModel(n_features, sequence_length, number_of_dependent_signals)
        self.criterion = nn.MSELoss()

    def forward(self, x, labels =None):
        output = self.model(x)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels.unsqueeze(dim=1))
        return loss, output   

    def training_step(self, batch, batch_idx):
        sequences = batch['sequence']
        labels = batch['label']

        loss, output = self(sequences, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        sequences = batch['sequence']
        labels = batch['label']

        loss, output = self(sequences, labels) #
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        sequences = batch['sequence']
        labels = batch['label']

        loss, output = self(sequences, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr = 0.0001)





In [6]:
geo_data = ['latitude_degree','longitude_degree']

independent_signals = ['accelerator_pedal','accelerator_pedal_gradient_sign','brake_pressure','steering_angle_calculated','steering_angle_calculated_sign']
dependent_signals = ['vehicle_speed','roll_angle','pitch_angle']

sequence_length = 10

N_EPOCHS = 100
BATCH_SIZE = 64



data_prep = DataPreprocessing('datasets/', independent_signals, dependent_signals, sequence_length,train_size=.75)
data_prep.fit_transform()

train_sequence = data_prep.get_sequences_train
test_sequence = data_prep.get_sequences_test

data_module = TorchDataModule(train_sequence, test_sequence, batch_size = BATCH_SIZE)
data_module.setup()

"""for item in data_module.train_dataloader(): 
    print(item['sequence'].shape)
    print(item['label'].shape)
    break"""

datasets/resampled_20180810150607.csv
datasets/resampled_20190401121727.csv
datasets/resampled_20190401145936.csv


"for item in data_module.train_dataloader(): \n    print(item['sequence'].shape)\n    print(item['label'].shape)\n    break"

In [4]:
%load_ext tensorboard
%tensorboard --logdir ./logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 1250), started 0:00:16 ago. (Use '!kill 1250' to kill it.)

In [10]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1, 
    verbose=True, 
    monitor="val_loss",
    mode="min")

logger = TensorBoardLogger("logs", name='LSTM')

early_stopping_callback = EarlyStopping(monitor='val_loss',
                                        patience=25)



In [11]:
trainer = pl.Trainer(
    logger=logger,
    callbacks=[early_stopping_callback, checkpoint_callback],
    max_epochs=N_EPOCHS
) 

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [12]:
model = Predictor(n_features=len(independent_signals),
                   sequence_length=sequence_length,
                   number_of_dependent_signals=len(dependent_signals))


In [1]:
import warnings
warnings.filterwarnings("ignore")
trainer.fit(model, data_module.train_dataloader(), data_module.val_dataloader())

NameError: name 'trainer' is not defined

In [18]:
"""for item in data_module.train_dataloader(): 
    print(item['sequence'].shape)
    print(item['label'].shape)
    print('------')
    break"""

"for item in data_module.train_dataloader(): \n    print(item['sequence'].shape)\n    print(item['label'].shape)\n    print('------')\n    break"